# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformer_lens import HookedTransformer
import random
import sys
import os
import collections
import operator

base_dir = os.path.split(os.getcwd())[0]
sys.path.append(base_dir)
from pyfunctions.cdt_basic import *
from pyfunctions.cdt_source_to_target import *
from pyfunctions.cdt_from_source_nodes import *
from pyfunctions.toy_model import *
from pyfunctions.faithfulness_ablations import add_mean_ablation_hook

import torch
Result = collections.namedtuple('Result', ('ablation_set', 'score'))


/raid/conda/envs/dacslab_djk_llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# from mi_utils_public import *
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
torch.autograd.set_grad_enabled(False)
model = HookedTransformer.from_pretrained('attn-only-4l',
                                         fold_ln=False)

"""Toggle whether to explicitly calculate and expose the result for each attention head.
Useful for interpretability but can easily burn through GPU memory.
        """
model.cfg.use_attn_result = False # Not actually needed because we have methods for accessing the activation cache anyway.

# note from the mi_utils guys: "because running subprocess breaks things"
# %env TOKENIZERS_PARALLELISM=true



/raid/conda/envs/dacslab_djk_llm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model attn-only-4l into HookedTransformer


In [4]:

from im_utils.prompts import Prompt, docstring_ind_prompt_gen

batch_size = 5000

docstring_ind_prompt_kwargs = dict(
    n_matching_args=3,
    n_def_prefix_args=2,
    n_def_suffix_args=1,
    n_doc_prefix_args=0,
    met_desc_len=3,
    arg_desc_len=2
)
prompts = [docstring_ind_prompt_gen("rest", **docstring_ind_prompt_kwargs) for _ in range(batch_size)]

In [5]:
tokens = model.to_str_tokens(prompts[0].clean_prompt)


In [6]:
pos_labels = model.to_str_tokens(prompts[0].clean_prompt)
pos_labels[ 0] = "BOS"
pos_labels[ 1] = "⏎"
pos_labels[ 2] = "def"
pos_labels[ 3] = "rand0"
pos_labels[ 4] = "("
pos_labels[ 5] = "self"
pos_labels[ 6] = ",_0"
pos_labels[ 7] = "rand1"
pos_labels[ 8] = ",_1"
pos_labels[ 9] = "rand2"
pos_labels[10] = ",_2"
pos_labels[11] = "A_def"
pos_labels[12] = ",_A"
pos_labels[13] = "B_def"
pos_labels[14] = ",_B"
pos_labels[15] = "C_def"
pos_labels[16] = ",_C"
pos_labels[17] = "rand3"
pos_labels[18] = "):"
pos_labels[19] = "⏎···_0"
pos_labels[20] = '"""'
pos_labels[21] = "rand4"
pos_labels[22] = "rand5"
pos_labels[23] = "rand6"
pos_labels[24] = "⏎⏎···_1"
pos_labels[25] = "·:_0"
pos_labels[26] = "param_1"
pos_labels[27] = "A_doc"
pos_labels[28] = ":_1"
pos_labels[29] = "rand7"
pos_labels[30] = "rand8"
pos_labels[31] = "⏎···_2"
pos_labels[32] = "·:_1"
pos_labels[33] = "param_2"
pos_labels[34] = "B_doc"
pos_labels[35] = ":_2"
pos_labels[36] = "rand9"
pos_labels[37] = "rand10"
pos_labels[38] = "⏎···_3"
pos_labels[39] = "·:_2"
pos_labels[40] = "param_3"

In [7]:
# Get a "better" dataset, i.e, ones where the model answers the correct answer, preferably by a wide margin.
# the toy model actually isn't very good at the docstring task, only getting it right like 50-75% of the time.
# We obviously can't investigate the question of how the model does the docstring task if it's not actually "doing" it.
good_prompts = []
for prompt in prompts:
    clean_prompt = prompt.clean_prompt
    tokens = model.to_str_tokens(clean_prompt)
    # it's perhaps better to check if the model's prediction logit is highest by a certain threshold.
    correct_answer = tokens[pos_labels.index("C_def")]
    gen_tokens = model.generate(clean_prompt, max_new_tokens=1, do_sample=False, return_type='tokens')
    model_prediction = model.to_string(gen_tokens[0, -1])
    if model_prediction == correct_answer:
        good_prompts.append(prompt)
prompts = good_prompts

100%|██████████| 1/1 [00:00<00:00, 206.05it/s]


In [8]:
len(prompts)

3138

In [9]:
num_layers = len(model.blocks)
seq_len = len(pos_labels)
num_attention_heads = model.cfg.n_heads

In [10]:
# this thing assumes the input is a list of sequences, the sequences are a list of tokens
# if you pass in only a single list, you should wrap it in a [] 
def compute_logits_decomposition_scores(texts, out_decomps, normalized=True):
    if not isinstance(texts, list):
        texts = [texts] # so that the batching works out
    relevances = np.zeros((num_layers, seq_len, num_attention_heads))
     # NOT to_tokens(text)
    batch_size = len(texts)
    results = []

    for layer_idx in range(num_layers):
        for seq_pos in range(seq_len):
            for head_idx in range(num_attention_heads):
                score = 0
                decomp = out_decomps[layer_idx * seq_len * num_attention_heads + seq_pos * num_attention_heads + head_idx]
                for i in range(batch_size):
                    tokens = model.to_str_tokens(texts[i])
                    correct_logit_idx = model.to_single_token(tokens[pos_labels.index("C_def")])
                    rel_correct_logit = decomp.rel[i, -1, correct_logit_idx]
                    incorrect_seq_idxs = [pos_labels.index("A_def"), pos_labels.index("B_def"),
                                     pos_labels.index("rand0"),
                                     pos_labels.index("rand1"),
                                     pos_labels.index("rand2"),
                                     pos_labels.index("rand3"),
                                     pos_labels.index("rand4"),
                                     pos_labels.index("rand5"),
                                     pos_labels.index("rand6"),
                                     pos_labels.index("rand7"),
                                     pos_labels.index("rand8"),
                                     pos_labels.index("rand9"),
                                     pos_labels.index("rand10"),
                                     ]
                    incorrect_logit_idxs = [model.to_single_token(tokens[idx]) for idx in incorrect_seq_idxs]
                    rel_incorrect_logits = decomp.rel[i, -1, incorrect_logit_idxs]
                    score += np.min(rel_correct_logit - rel_incorrect_logits)
                    relevances[layer_idx, seq_pos, head_idx] = score
                score /= batch_size
                if not normalized:
                    results.append(Result(decomp.ablation_set, score))
    if normalized:
        sums_per_layer = np.sum(np.abs(relevances), axis=(1, 2))
        sums_per_layer[sums_per_layer == 0] = -1e-8
        relevances = relevances / np.expand_dims(sums_per_layer, (1, 2))
        for layer_idx in range(num_layers):
            for seq_pos in range(seq_len):
                for head_idx in range(num_attention_heads):
                    decomp = out_decomps[layer_idx * seq_len * num_attention_heads + seq_pos * num_attention_heads + head_idx]
                    results.append(Result(decomp.ablation_set, relevances[layer_idx, seq_pos, head_idx]))
    results.sort(key=operator.attrgetter('score'), reverse=True)
    return results, relevances

In [11]:
def calculate_target_decomposition_scores(target_decomps, normalized=False):
    relevances = np.zeros((num_layers, seq_len, num_attention_heads))
    results = []
    for layer_idx in range(num_layers):
        for seq_pos in range(seq_len):
            for head_idx in range(num_attention_heads):
                score = 0
                target_decomp = target_decomps[layer_idx * seq_len * num_attention_heads + seq_pos * num_attention_heads + head_idx]

                for i in range(len(target_decomp.target_nodes)):
                    rels_magnitude = torch.mean(abs(target_decomp.rels[i])) # np.mean if you are on cpu
                    irrels_magnitude = torch.mean(abs(target_decomp.irrels[i])) # np.mean if you are on cpu
                    target_node_score = rels_magnitude / (rels_magnitude + irrels_magnitude)
                    score += (target_node_score)
                if target_decomp.ablation_set[0] in target_nodes:
                    score = 0
                relevances[layer_idx, seq_pos, head_idx] = score
                if not normalized:
                    results.append(Result(target_decomp.ablation_set, relevances[layer_idx, seq_pos, head_idx]))
    if normalized:
        sums_per_layer = np.sum(relevances, axis=(1, 2))
        sums_per_layer[sums_per_layer == 0] = -1e-8
        relevances = relevances / np.expand_dims(sums_per_layer, (1, 2))

        for layer_idx in range(num_layers):
            for seq_pos in range(seq_len):
                for head_idx in range(num_attention_heads):
                    target_decomp = target_decomps[layer_idx * seq_len * num_attention_heads + seq_pos * num_attention_heads + head_idx]
                    results.append(Result(target_decomp.ablation_set, relevances[layer_idx, seq_pos, head_idx]))
    
    results.sort(key=operator.attrgetter('score'), reverse=True)
    return results, relevances

# Datatype investigation

In [12]:
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-3): 4 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_post): HookPoint()
    )
  )
  (ln_final): LayerNorm(
    (hook_scale): HookPoint()
    (hook_normalized): HookPoint()
  )
  (unembed): Unembed()
)


In [13]:
# dir(prompts[0])
'''
'clean_prompt',
 'correct_answers',
 'corrupt_prompt',
 'print_all_corrupt',
 'print_all_corrupt_tokenized',
 'print_clean',
 'print_clean_tokenized',
 'print_corrupt',
 'print_corrupt_tokenized',
 'print_tokenized',
 'wrong_answers'
'''

print(prompts[0].clean_prompt)
# 'random_doc', 'random_def', 'random_answer', 'random_def_doc', 'random_answer_doc', 'random_random', 'vary_length_doc_desc', 'vary_length_doc_desc_random_doc'
# print(prompts[0].corrupt_prompt.keys())
# type(prompts[0])
# These are actually decently well explained in the original LessWrong post, I just didn't see it at first.
print(prompts[0].corrupt_prompt['random_doc'])
print(prompts[0].corrupt_prompt['random_def_doc'])
print(prompts[0].corrupt_prompt['random_answer_doc'])
print(prompts[0].corrupt_prompt['random_random'])
print(prompts[0].corrupt_prompt['vary_length_doc_desc'])
print(prompts[0].corrupt_prompt['vary_length_doc_desc_random_doc'])



def query(self, instance, names, shape, header, expected, item):
    """lake fire register

    :param shape: hair frame
    :param header: bed sample
    :param

def query(self, instance, names, shape, header, expected, item):
    """lake fire register

    :param value: hair frame
    :param model: bed sample
    :param

def query(self, instance, names, value, model, expected, item):
    """lake fire register

    :param value: hair frame
    :param model: bed sample
    :param

def query(self, instance, names, shape, header, data, item):
    """lake fire register

    :param value: hair frame
    :param model: bed sample
    :param

def query(self, instance, names, default, url, name, item):
    """lake fire register

    :param line: hair frame
    :param server: bed sample
    :param

def query(self, instance, names, shape, header, expected, item):
    """lake fire register

    :param shape: hair frame
    :param header: bed sample
    :param

def query(self, instance, names, sh

# Experiments

In [14]:
'''
Summary of docstring post's results:
Arg movers: 3.0, 3.6 at last position (40)
Induction: 1.4 at last position (40)
Prev token: 0.2 at B_doc, or the one after? (34) or (35) # can't be found by patching, maybe can be found here
Prev token: 2.0 at C_def (15)
Prev token: 1.4 at the fourth comma?? (14)

Also 0.5 at B_def (13), C_def (15) and B_doc (34), (0.4 relevant to 1.4), 1.2.
Also 2.3, 0.5, 1.2, 0.0, 0.1, 2.2, in roughly that order.
'''

"\nSummary of docstring post's results:\nArg movers: 3.0, 3.6 at last position (40)\nInduction: 1.4 at last position (40)\nPrev token: 0.2 at B_doc, or the one after? (34) or (35) # can't be found by patching, maybe can be found here\nPrev token: 2.0 at C_def (15)\nPrev token: 1.4 at the fourth comma?? (14)\n\nAlso 0.5 at B_def (13), C_def (15) and B_doc (34), (0.4 relevant to 1.4), 1.2.\nAlso 2.3, 0.5, 1.2, 0.0, 0.1, 2.2, in roughly that order.\n"

In [15]:

# Visualization of task
prompt_idx = 0
example_prompt = prompts[prompt_idx].clean_prompt
tokens = model.to_str_tokens(example_prompt)

example_answer = tokens[pos_labels.index("C_def")]
print(example_prompt)
print(example_answer)
transformer_lens.utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)


def query(self, instance, names, shape, header, expected, item):
    """lake fire register

    :param shape: hair frame
    :param header: bed sample
    :param
 expected
Tokenized prompt: ['<|BOS|>', '\n', 'def', ' query', '(', 'self', ',', ' instance', ',', ' names', ',', ' shape', ',', ' header', ',', ' expected', ',', ' item', '):', '\n   ', ' """', 'lake', ' fire', ' register', '\n\n   ', ' :', 'param', ' shape', ':', ' hair', ' frame', '\n   ', ' :', 'param', ' header', ':', ' bed', ' sample', '\n   ', ' :', 'param']
Tokenized answer: [' expected']


Performance on answer token:
Rank: 0        Logit: 18.46 Prob: 37.91% Token: | expected|

Top 0th token. Logit: 18.46 Prob: 37.91% Token: | expected|
Top 1th token. Logit: 17.29 Prob: 11.68% Token: | header|
Top 2th token. Logit: 16.16 Prob:  3.77% Token: | name|
Top 3th token. Logit: 15.84 Prob:  2.74% Token: | body|
Top 4th token. Logit: 15.13 Prob:  1.35% Token: | shape|
Top 5th token. Logit: 15.12 Prob:  1.33% Token: | attributes|
Top 6th token. Logit: 15.06 Prob:  1.26% Token: | type|
Top 7th token. Logit: 15.06 Prob:  1.26% Token: | value|
Top 8th token. Logit: 14.67 Prob:  0.85% Token: | size|
Top 9th token. Logit: 14.58 Prob:  0.78% Token: | file|


Ranks of the answer tokens: [(' expected', 0)]

In [16]:
import itertools
import functools



model.reset_hooks(including_permanent=True)

NUM_SAMPLES = 32
NUM_OUTLIERS_TO_KEEP_PER_ITER = 2

corrupt_prompts = [prompt.corrupt_prompt['random_answer_doc'] for prompt in prompts[:NUM_SAMPLES]]

corrupt_logits, corrupt_cache = model.run_with_cache(corrupt_prompts)
corrupt_attention_outputs = [corrupt_cache['blocks.' + str(i) + '.attn.hook_z'] for i in range(num_layers)]
corrupt_attention_outputs = torch.stack(corrupt_attention_outputs, dim=1).squeeze(0) # now batch, layer, seq, n_heads, dim_attn (squeezed away the batch dim)
old_shape = corrupt_attention_outputs.shape
last_dim = old_shape[-2] * old_shape[-1]
new_shape = old_shape[:-2] + (last_dim,)
corrupt_attention_outputs = corrupt_attention_outputs.view(new_shape) 

encoding_idxs = model.tokenizer.batch_encode_plus([prompt.clean_prompt for prompt in prompts], 
                                 add_special_tokens=True, 
                                 max_length=512,
                                 truncation=True, 
                                 padding = "longest", 
                                 return_attention_mask=True, 
                                 return_tensors="pt").to(device).input_ids
# put in only one example for the attention mask, so that it has batch dimension 1, which broadcasts
attention_mask = model.tokenizer.encode_plus(prompts[0].clean_prompt, 
                                 add_special_tokens=True, 
                                 max_length=512,
                                 truncation=True, 
                                 padding = "longest", 
                                 return_attention_mask=True, 
                                 return_tensors="pt").to(device).attention_mask
# if we want to make HF prepend the BOS token, which this model expects
# so do it ourselves manually
attention_mask = torch.tensor([[1] + attention_mask.cpu().numpy().tolist()[0]], device=device)
encoding_idxs = torch.tensor(np.hstack([np.ones((encoding_idxs.shape[0], 1), dtype=np.int64), encoding_idxs.cpu().numpy()]), device=device)

input_shape = attention_mask.size()
extended_attention_mask = get_extended_attention_mask(attention_mask, 
                                                        input_shape, 
                                                        model,
                                                        device)

# Calculate relevance to logits
'''
ranges = [
        [layer for layer in range(num_layers)],
        [sequence_position for sequence_position in range(seq_len)],
        [attention_head_idx for attention_head_idx in range(num_attention_heads)]
    ]

source_nodes = [Node(*x) for x in itertools.product(*ranges)]
ablation_sets = [(n,) for n in source_nodes]
'''
ablation_sets = []
for layer in range(num_layers):
    for head_idx in range(num_attention_heads):
        ablation_sets.append(tuple(Node(layer, seq_pos, head_idx) for seq_pos in range(seq_len)))
target_nodes = []

# cache activations for faster batch run
out_decomp, _, _, pre_layer_activations = prop_toy_model_4l(encoding_idxs[0:NUM_SAMPLES, :], extended_attention_mask, model, [ablation_sets[0]], target_nodes=target_nodes, device=device, mean_acts=corrupt_attention_outputs, set_irrel_to_mean=True)
prop_fn = lambda ablation_list: prop_toy_model_4l(encoding_idxs[0:NUM_SAMPLES, :], extended_attention_mask, model, ablation_list, target_nodes=target_nodes, \
                                                  device=device, mean_acts=corrupt_attention_outputs, set_irrel_to_mean=True, cached_pre_layer_acts=pre_layer_activations)
out_decomps, _ = batch_run(prop_fn, ablation_sets, num_at_time=(64 // NUM_SAMPLES))


/raid/conda/envs/dacslab_djk_llm/lib/python3.11/site-packages/transformers/modeling_utils.py:1080: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Running decomposition in batches...: 100%|██████████| 16/16 [00:06<00:00,  2.58it/s]


In [17]:
# modified to not worry abouy seq len
def compute_logits_decomposition_scores(texts, out_decomps, normalized=True):
    if not isinstance(texts, list):
        texts = [texts] # so that the batching works out
    relevances = np.zeros((num_layers, num_attention_heads))
    batch_size = len(texts)
    results = []

    for layer_idx in range(num_layers):
        for head_idx in range(num_attention_heads):
            score = 0
            decomp = out_decomps[layer_idx * num_attention_heads + head_idx]
            for i in range(batch_size):
                tokens = model.to_str_tokens(texts[i])
                correct_logit_idx = model.to_single_token(tokens[pos_labels.index("C_def")])
                rel_correct_logit = decomp.rel[i, -1, correct_logit_idx]
                incorrect_seq_idxs = [pos_labels.index("A_def"), pos_labels.index("B_def"),
                                 pos_labels.index("rand0"),
                                 pos_labels.index("rand1"),
                                 pos_labels.index("rand2"),
                                 pos_labels.index("rand3"),
                                 pos_labels.index("rand4"),
                                 pos_labels.index("rand5"),
                                 pos_labels.index("rand6"),
                                 pos_labels.index("rand7"),
                                 pos_labels.index("rand8"),
                                 pos_labels.index("rand9"),
                                 pos_labels.index("rand10"),
                                 ]
                incorrect_logit_idxs = [model.to_single_token(tokens[idx]) for idx in incorrect_seq_idxs]
                rel_incorrect_logits = decomp.rel[i, -1, incorrect_logit_idxs]
                score += np.min(rel_correct_logit - rel_incorrect_logits)
                relevances[layer_idx, head_idx] = score
            score /= batch_size
            if not normalized:
                results.append(Result(decomp.ablation_set, score))
    if normalized:
        sums_per_layer = np.sum(np.abs(relevances), axis=(1,))
        sums_per_layer[sums_per_layer == 0] = -1e-8
        relevances = relevances / np.expand_dims(sums_per_layer, (1))
        for layer_idx in range(num_layers):
            for head_idx in range(num_attention_heads):
                decomp = out_decomps[layer_idx * num_attention_heads + head_idx]
                results.append(Result(decomp.ablation_set, relevances[layer_idx, head_idx]))
    results.sort(key=operator.attrgetter('score'), reverse=True)
    return results, relevances

In [18]:

results, relevances = compute_logits_decomposition_scores([prompt.clean_prompt for prompt in prompts[0:NUM_SAMPLES]], out_decomps)# text, out_decomps)
for result in results[:20]:
    print(result.ablation_set[0], result.score)

Node(layer_idx=3, sequence_idx=0, attn_head_idx=0) 0.3308493685856119
Node(layer_idx=3, sequence_idx=0, attn_head_idx=6) 0.27684701730886785
Node(layer_idx=2, sequence_idx=0, attn_head_idx=3) -0.009781594487445557
Node(layer_idx=3, sequence_idx=0, attn_head_idx=4) -0.0333818470753158
Node(layer_idx=3, sequence_idx=0, attn_head_idx=2) -0.04348649948658609
Node(layer_idx=3, sequence_idx=0, attn_head_idx=3) -0.04719218616503724
Node(layer_idx=3, sequence_idx=0, attn_head_idx=1) -0.05933870252091714
Node(layer_idx=1, sequence_idx=0, attn_head_idx=3) -0.06129851936207037
Node(layer_idx=1, sequence_idx=0, attn_head_idx=6) -0.07185441473281527
Node(layer_idx=3, sequence_idx=0, attn_head_idx=5) -0.07249264523787288
Node(layer_idx=1, sequence_idx=0, attn_head_idx=7) -0.09233366562959222
Node(layer_idx=2, sequence_idx=0, attn_head_idx=1) -0.0962539615877492
Node(layer_idx=1, sequence_idx=0, attn_head_idx=5) -0.10081785810593039
Node(layer_idx=0, sequence_idx=0, attn_head_idx=7) -0.10729969798731

In [19]:

ablation_sets = []
for layer in range(num_layers):
    for head_idx in range(num_attention_heads):
        ablation_sets.append(tuple(Node(layer, seq_pos, head_idx) for seq_pos in range(seq_len)))
target_nodes = []
for layer, head_idx in [(3, 0), (3, 6)]:
    for seq_pos in range(seq_len):
        target_nodes.append(Node(layer, seq_pos, head_idx))
        
prop_fn = lambda ablation_list: prop_toy_model_4l(
    encoding_idxs[0:NUM_SAMPLES, :],
    extended_attention_mask,
    model,
    ablation_list,
    target_nodes=target_nodes,
    device=device,
    mean_acts=corrupt_attention_outputs,
    set_irrel_to_mean=True,
    cached_pre_layer_acts=pre_layer_activations,
    return_out_decomps=False,
)
_, target_decomps = batch_run(prop_fn, ablation_sets, num_at_time=(64 // NUM_SAMPLES))


Running decomposition in batches...:   0%|          | 0/16 [00:00<?, ?it/s]

Running decomposition in batches...: 100%|██████████| 16/16 [00:00<00:00, 23.36it/s]


In [20]:
# modified to not worry about seq len
def calculate_target_decomposition_scores(target_decomps, normalized=False):
    relevances = np.zeros((num_layers, num_attention_heads))
    results = []
    for layer_idx in range(num_layers):

            for head_idx in range(num_attention_heads):
                score = 0
                target_decomp = target_decomps[layer_idx * num_attention_heads + head_idx]

                for i in range(len(target_decomp.target_nodes)):
                    rels_magnitude = torch.mean(abs(target_decomp.rels[i])) # np.mean if you are on cpu
                    irrels_magnitude = torch.mean(abs(target_decomp.irrels[i])) # np.mean if you are on cpu
                    target_node_score = rels_magnitude / (rels_magnitude + irrels_magnitude)
                    score += (target_node_score)
                if target_decomp.ablation_set[0] in target_nodes:
                    score = 0
                relevances[layer_idx, head_idx] = score
                if not normalized:
                    results.append(Result(target_decomp.ablation_set, relevances[layer_idx, head_idx]))
    if normalized:
        sums_per_layer = np.sum(np.abs(relevances), axis=(1))
        sums_per_layer[sums_per_layer == 0] = -1e-8
        relevances = relevances / np.expand_dims(sums_per_layer, (1))

        for layer_idx in range(num_layers):
            for head_idx in range(num_attention_heads):
                target_decomp = target_decomps[layer_idx * num_attention_heads + head_idx]
                results.append(Result(target_decomp.ablation_set, relevances[layer_idx, head_idx]))

    results.sort(key=operator.attrgetter('score'), reverse=True)
    return results, relevances

In [21]:

results, relevances = calculate_target_decomposition_scores(target_decomps, normalized=True)

for result in results[:20]:
    print(result.ablation_set[0])

Node(layer_idx=2, sequence_idx=0, attn_head_idx=0)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=0)
Node(layer_idx=2, sequence_idx=0, attn_head_idx=3)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=0)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=4)
Node(layer_idx=2, sequence_idx=0, attn_head_idx=2)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=5)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=1)
Node(layer_idx=2, sequence_idx=0, attn_head_idx=5)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=1)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=6)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=2)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=7)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=3)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=7)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=5)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=2)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=3)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=6)
Node(layer_idx=2, sequence_idx=

In [22]:


# Now, find maximally relevant source nodes to target nodes

outliers_per_iter = []
while True:
    outliers = results[:NUM_OUTLIERS_TO_KEEP_PER_ITER]
    outliers_per_iter.append(outliers)
    target_nodes = [r.ablation_set[0] for r in outliers]
    print(target_nodes)
    should_break = True
    for node in target_nodes:
        if node.layer_idx != 0:
            should_break = False
    if should_break:
        break
    
    ranges = [
            [layer for layer in range(num_layers)],
            [sequence_position for sequence_position in range(seq_len)],
            # [ioi_dataset.word_idx['IO'][0]],
            [attention_head_idx for attention_head_idx in range(num_attention_heads)]
        ]
    source_nodes = [Node(*x) for x in itertools.product(*ranges)]
    ablation_sets = [(n,) for n in source_nodes]
    prop_fn = lambda ablation_list: prop_toy_model_4l(
        encoding_idxs[0:NUM_SAMPLES, :],
        extended_attention_mask,
        model,
        ablation_list,
        target_nodes=target_nodes,
        device=device,
        mean_acts=corrupt_attention_outputs,
        set_irrel_to_mean=True,
        cached_pre_layer_acts=pre_layer_activations,
        return_out_decomps=False,
    )
    _, target_decomps = batch_run(prop_fn, ablation_sets, num_at_time=(64 // NUM_SAMPLES))
    
    results, relevances = calculate_target_decomposition_scores(target_decomps)


[Node(layer_idx=2, sequence_idx=0, attn_head_idx=0), Node(layer_idx=1, sequence_idx=0, attn_head_idx=0)]


Running decomposition in batches...: 100%|██████████| 656/656 [00:02<00:00, 275.61it/s]

[Node(layer_idx=0, sequence_idx=0, attn_head_idx=4), Node(layer_idx=0, sequence_idx=0, attn_head_idx=3)]


In [23]:
all_nodes = []
for it in outliers_per_iter:
    for result in it:
        if result.ablation_set[0] not in all_nodes:
            all_nodes.append(result.ablation_set[0])
for node in all_nodes:
    print((node))

Node(layer_idx=2, sequence_idx=0, attn_head_idx=0)
Node(layer_idx=1, sequence_idx=0, attn_head_idx=0)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=4)
Node(layer_idx=0, sequence_idx=0, attn_head_idx=3)


## Circuit evaluation

In [24]:

def correctness_rate(logits, test_dataset):
    tokens = model.to_str_tokens(test_dataset) # NOT to_tokens(text)
    correct_logits = np.array([model.to_single_token(tokens[i][pos_labels.index("C_def")]) for i in range(len(test_dataset))])
    # correct_logits = logits[list(range(len(test_dataset))), -1, correct_logit_idxs]
    predicted_logits = np.argmax(logits.cpu().numpy()[:, -1, :], axis=-1)
    return np.sum(correct_logits == predicted_logits)

In [25]:
def logits_to_ave_logit_diff(logits, test_dataset):
    total = 0
    for i in range(len(test_dataset)):
        text = test_dataset[i]
        tokens = model.to_str_tokens(text) # NOT to_tokens(text)
        correct_logit_idx = model.to_single_token(tokens[pos_labels.index("C_def")])
        rel_correct_logit = logits[i, -1, correct_logit_idx]
        incorrect_seq_idxs = [pos_labels.index("A_def"), pos_labels.index("B_def"),
                         pos_labels.index("rand0"),
                         pos_labels.index("rand1"),
                         pos_labels.index("rand2"),
                         pos_labels.index("rand3"),
                         pos_labels.index("rand4"),
                         pos_labels.index("rand5"),
                         pos_labels.index("rand6"),
                         pos_labels.index("rand7"),
                         pos_labels.index("rand8"),
                         pos_labels.index("rand9"),
                         pos_labels.index("rand10"),
                         ]
        incorrect_logit_idxs = [model.to_single_token(tokens[idx]) for idx in incorrect_seq_idxs]
        rel_incorrect_logits = logits[0, -1, incorrect_logit_idxs]
        total += np.min(rel_correct_logit - rel_incorrect_logits)
    avg = total / len(test_dataset)
    return avg

In [27]:
model.reset_hooks(including_permanent=True)

batch_size = 200

docstring_ind_prompt_kwargs = dict(
    n_matching_args=3,
    n_def_prefix_args=2,
    n_def_suffix_args=1,
    n_doc_prefix_args=0,
    met_desc_len=3,
    arg_desc_len=2
)
prompts = [docstring_ind_prompt_gen("rest", **docstring_ind_prompt_kwargs) for _ in range(batch_size)]

# Get a "better" dataset, i.e, ones where the model answers the correct answer, preferably by a wide margin.
# the toy model actually isn't very good at the docstring task, only getting it right like 50-75% of the time.
# We obviously can't investigate the question of how the model does the docstring task if it's not actually "doing" it.
good_prompts = []
for prompt in prompts:
    clean_prompt = prompt.clean_prompt
    tokens = model.to_str_tokens(clean_prompt)
    # it's perhaps better to check if the model's prediction logit is highest by a certain threshold.
    correct_answer = tokens[pos_labels.index("C_def")]
    gen_tokens = model.generate(clean_prompt, max_new_tokens=1, do_sample=False, return_type='tokens')
    model_prediction = model.to_string(gen_tokens[0, -1])
    if model_prediction == correct_answer:
        good_prompts.append(prompt)
prompts = good_prompts
print(len(prompts))


TEST_DATASET_SIZE = 100
test_dataset = [prompt.clean_prompt for prompt in prompts[:TEST_DATASET_SIZE:]]

corrupt_prompts = [prompt.corrupt_prompt['random_answer_doc'] for prompt in prompts[:TEST_DATASET_SIZE]]

corrupt_logits, corrupt_cache = model.run_with_cache(corrupt_prompts)
corrupt_attention_outputs = [corrupt_cache['blocks.' + str(i) + '.attn.hook_z'] for i in range(num_layers)]
corrupt_attention_outputs = torch.stack(corrupt_attention_outputs, dim=0) # now batch, layer, seq_ nheads, dim_model

100%|██████████| 1/1 [00:00<00:00, 206.89it/s]


120


In [28]:
model.reset_hooks(including_permanent=True)
logits, cache = model.run_with_cache(test_dataset) # full model performance
ave_logit_diff = logits_to_ave_logit_diff(logits.cpu().numpy(), test_dataset)
print(ave_logit_diff)
print(correctness_rate(logits, test_dataset))

4.593232307434082
100


In [29]:
random_circuit = random.sample(source_nodes, 20)
model.reset_hooks(including_permanent=True)
model = add_mean_ablation_hook(model, patch_values=corrupt_attention_outputs, circuit=random_circuit) # baseline 
logits, cache = model.run_with_cache(test_dataset) # run on entire dataset along batch dimension
ave_logit_diff = logits_to_ave_logit_diff(logits.cpu().numpy(), test_dataset)
print(ave_logit_diff)
print(correctness_rate(logits, test_dataset))

-2.0585064697265625
0


In [30]:

circuit = []
for (layer_idx, head_idx) in [(3, 0), (3, 6), (1, 4), (0, 5), (0, 0), (1, 0), (1, 4), (0, 1), (2, 3), (1, 2)]: # this is our finding
    for seq_pos in range(seq_len):
        circuit.append(Node(layer_idx, seq_pos, head_idx))
model.reset_hooks(including_permanent=True)
model = add_mean_ablation_hook(model, patch_values=corrupt_attention_outputs, circuit=circuit)
logits, cache = model.run_with_cache(test_dataset) # run on entire dataset along batch dimension
ave_logit_diff = logits_to_ave_logit_diff(logits.cpu().numpy(), test_dataset)
model.reset_hooks(including_permanent=True)

print(ave_logit_diff)
print(correctness_rate(logits, test_dataset))

3.5562384605407713
57


In [31]:

# 0.0 and 2.2 are negative in impact
# docstring project result
circuit = []
for (layer_idx, head_idx) in [(0, 2), (0, 4), (0, 5), (1, 2), (1, 4), (2, 0), (3, 0), (3, 6)]: 
    for seq_pos in range(seq_len):
        circuit.append(Node(layer_idx, seq_pos, head_idx))
model.reset_hooks(including_permanent=True)
model = add_mean_ablation_hook(model, patch_values=corrupt_attention_outputs, circuit=circuit)
logits, cache = model.run_with_cache(test_dataset) # run on entire dataset along batch dimension
ave_logit_diff = logits_to_ave_logit_diff(logits.cpu().numpy(), test_dataset)
print(ave_logit_diff)
print(correctness_rate(logits, test_dataset))
model.reset_hooks(including_permanent=True)


# docstring project + (i.e, they found some heads by means external to their analysis)
circuit = []

for (layer_idx, head_idx) in [(0, 2), (0, 4), (0, 5), (1, 2), (1, 4), (2, 0), (3, 0), (3, 6), (1, 0)]:
    for seq_pos in range(seq_len):
        circuit.append(Node(layer_idx, seq_pos, head_idx))
model.reset_hooks(including_permanent=True)
model = add_mean_ablation_hook(model, patch_values=corrupt_attention_outputs, circuit=circuit)
logits, cache = model.run_with_cache(test_dataset) # run on entire dataset along batch dimension
ave_logit_diff = logits_to_ave_logit_diff(logits.cpu().numpy(), test_dataset)
print(ave_logit_diff)
print(correctness_rate(logits, test_dataset))
model.reset_hooks(including_permanent=True)

# docstring project ++ (i.e, they found some more heads by means external to their analysis; this is the best circuit they could find by any means)
circuit = []

for (layer_idx, head_idx) in [(0, 2), (0, 4), (0, 5), (1, 2), (1, 4), (2, 0), (3, 0), (3, 6), (1, 0), (0, 1), (2, 3)]:
    for seq_pos in range(seq_len):
        circuit.append(Node(layer_idx, seq_pos, head_idx))
model.reset_hooks(including_permanent=True)
model = add_mean_ablation_hook(model, patch_values=corrupt_attention_outputs, circuit=circuit)
logits, cache = model.run_with_cache(test_dataset) # run on entire dataset along batch dimension
ave_logit_diff = logits_to_ave_logit_diff(logits.cpu().numpy(), test_dataset)
model.reset_hooks(including_permanent=True)
print(ave_logit_diff)
print(correctness_rate(logits, test_dataset))

2.7748719882965087
57
2.766711349487305
53
3.6008393001556396
62
